In [3]:
%load_ext autoreload
%autoreload 2

from lib.ekyn import *
import matplotlib.pyplot as plt
from lib.models import MLP
from torch.utils.data import DataLoader,ConcatDataset
from lib.deep_learning_utils import count_params
import torch
import os
from lib.env import DATA_PATH
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from lib.deep_learning_utils import training_loop,development_loop

if not os.path.isdir(f'{DATA_PATH}/projects'):
    os.makedirs(f'{DATA_PATH}/projects')

existing_projects = sorted([int(dir) for dir in os.listdir(f'{DATA_PATH}/projects')])
if len(existing_projects) == 0:
    PROJECT_ID = 0
else:
    PROJECT_ID = existing_projects[-1] + 1

PROJECT_PATH = f'{DATA_PATH}/projects/{PROJECT_ID}'
if not os.path.isdir(PROJECT_PATH):
    os.makedirs(PROJECT_PATH)

CONFIG = {
    'BATCH_SIZE':512,
    'LEARNING_RATE':3e-4,
    'LRS':[],
    'WEIGHT_DECAY':1e-4,
    'TRAIN_IDS':[],
    'TEST_IDS':[],
    'TRAINLOSS':[],
    'TRAINF1':[],
    'DEVLOSS':[],
    'DEVF1':[],
    'BEST_DEV_LOSS':torch.inf,
    'BEST_DEV_F1':0,
    'BEST_DEV_LOSS_EPOCH':0,
    'BEST_DEV_F1_EPOCH':0,  
    'LAST_EPOCH':0,
    'PROGRESS':0,
    'PATIENCE':100,
    'DEVICE':'cuda'
}

CONFIG['TRAIN_IDS'],CONFIG['TEST_IDS'] = train_test_split(get_ekyn_ids()[:2],test_size=.25,random_state=0)
print(f'Creating project {PROJECT_ID}')
print(f'Training ids: {CONFIG["TRAIN_IDS"]}')
print(f'Testing ids: {CONFIG["TEST_IDS"]}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Creating project 68
Training ids: ['A1-0']
Testing ids: ['A1-1']


In [4]:
from lib.models import ResidualBlock
from torch import nn
from torch.nn.functional import relu

class ResidualBlock(nn.Module):
    
    def __init__(self,in_feature_maps,out_feature_maps) -> None:
        super().__init__()
        self.c1 = nn.Conv1d(in_feature_maps,out_feature_maps,kernel_size=3,padding=3//2,bias=False)
        self.ln1 = nn.LayerNorm(out_feature_maps,elementwise_affine=False)

        self.c4 = nn.Conv1d(in_feature_maps,out_feature_maps,1,padding=0,bias=False)
        self.ln4 = nn.LayerNorm(out_feature_maps,elementwise_affine=False)

    def forward(self,x):
        identity = x
        x = self.c1(x)
        x = self.ln1(x.permute(0, 2, 1)).permute(0, 2, 1)  # LayerNorm expects [batch_size, length, channels]
        x = relu(x)

        identity = self.c4(identity)
        identity = self.ln4(identity.permute(0, 2, 1)).permute(0, 2, 1)

        x = x+identity
        x = relu(x)
        
        return x

class FrodoSmall(nn.Module):
    """
    the little wanderer
    """
    def __init__(self) -> None:
        super().__init__()
        self.block1 = ResidualBlock(1,4)
        self.block2 = ResidualBlock(4,8)
        self.block3 = ResidualBlock(8,8)
        
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(in_features=8,out_features=3)
    def forward(self,x,classification=True):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.gap(x)
        if(classification):
            x = self.fc1(x.squeeze())
            return x
        else:
            return x.squeeze()
    def get_dataloaders(self,batch_size,train_ids,test_ids):
        trainloader = DataLoader(
            dataset=ConcatDataset(
            [EpochedDataset(id=id,condition=condition) for id in train_ids for condition in CONDITIONS]
            ),
            batch_size=batch_size,
            shuffle=True,
            num_workers=0
        )
        testloader = DataLoader(
            dataset=ConcatDataset(
            [EpochedDataset(id=id,condition=condition) for id in test_ids for condition in CONDITIONS]
            ),
            batch_size=batch_size,
            shuffle=False,
            num_workers=0
        )
        print(f'{len(trainloader)*batch_size} training samples')
        print(f'{len(testloader)*batch_size} testing samples')
        return trainloader,testloader
    

model = FrodoSmall()
trainloader,testloader = model.get_dataloaders(CONFIG['BATCH_SIZE'],CONFIG['TRAIN_IDS'],CONFIG['TEST_IDS'])
model = torch.compile(model)
torch.set_float32_matmul_precision('high')
print(f'{count_params(model)} parameters')
criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor([100,1,1]))
optimizer = torch.optim.AdamW(model.parameters(),lr=CONFIG["LEARNING_RATE"],weight_decay=CONFIG["WEIGHT_DECAY"])
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)
model.to(CONFIG['DEVICE'])
criterion.to(CONFIG['DEVICE'])

Xi,yi = next(iter(trainloader))
print(Xi.shape,yi.shape)
Xi,yi = Xi.cuda(),yi.cuda()
# model(Xi)

17408 training samples
17408 testing samples


/home/andrew/sleep/sleep_ml/.env/lib/python3.11/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


427 parameters


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [4]:
Xi,yi = next(iter(trainloader))

In [7]:
yi.argmax(axis=1)

tensor([2, 0, 1, 1, 0, 1, 2, 1, 1, 1, 2, 1, 2, 0, 2, 2, 2, 1, 1, 1, 1, 1, 2, 0,
        2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 0, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1,
        0, 0, 0, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2,
        2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1,
        1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 0, 1, 1, 0, 2, 2, 2, 1, 1, 2, 1,
        2, 1, 2, 1, 2, 1, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 0,
        2, 2, 1, 2, 0, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1,
        2, 1, 2, 0, 2, 2, 0, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1,
        2, 2, 1, 1, 2, 2, 0, 2, 1, 0, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 0,
        0, 2, 0, 2, 1, 0, 1, 1, 1, 1, 2, 0, 2, 1, 2, 1, 2, 1, 0, 1, 2, 2, 1, 1,
        1, 0, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 0, 1, 1,
        1, 0, 2, 1, 1, 2, 2, 2, 2, 0, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1,
        1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1,

In [3]:
lossi = []
trainlossi = []
trainf1 = []
devlossi = []
devf1 = []
model.train()

for i in tqdm(range(1000)):
    loss,f1 = training_loop(model=model,trainloader=trainloader,criterion=criterion,optimizer=optimizer,device='cuda')
    trainlossi.append(loss)
    trainf1.append(f1)

    loss,f1 = development_loop(model=model,devloader=testloader,criterion=criterion,device='cuda')
    devlossi.append(loss)
    devf1.append(f1)
    # scheduler.step(loss)
    # CONFIG['LRS'].append(scheduler.get_last_lr())
    fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(13,4),dpi=200)
    ax[0].plot(trainlossi)
    ax[0].plot(devlossi)
    ax[0].set_title(label='loss',fontweight='bold')
    ax[0].set_xlabel('Epoch')
    ax[1].plot(trainf1)
    ax[1].plot(devf1)
    ax[1].set_title(label='f1',fontweight='bold')
    ax[1].set_xlabel('Epoch')

    plt.savefig('loss.jpg')
    plt.close()

  0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [ ]:
torch.save(model.state_dict(),f'{PROJECT_PATH}/model.last.pt')

In [ ]:
from lib.deep_learning_utils import evaluate
from sklearn.metrics import ConfusionMatrixDisplay,classification_report

loss,report,y_true,y_pred,y_logits = evaluate(dataloader=devloader,model=model,criterion=criterion)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='true')
plt.savefig(f'{PROJECT_PATH}/cm.jpg')
print(classification_report(y_true,y_pred))
print(loss)